<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-RDD-사용하기" data-toc-modified-id="1.-RDD-사용하기-1">1. RDD 사용하기</a></span><ul class="toc-item"><li><span><a href="#1)-map" data-toc-modified-id="1)-map-1.1">1) map</a></span><ul class="toc-item"><li><span><a href="#--csv파일에서-읽어-RDD-생성" data-toc-modified-id="--csv파일에서-읽어-RDD-생성-1.1.1">- csv파일에서 읽어 RDD 생성</a></span></li></ul></li><li><span><a href="#2)-filter()로-문장-분리" data-toc-modified-id="2)-filter()로-문장-분리-1.2">2) filter()로 문장 분리</a></span><ul class="toc-item"><li><span><a href="#-filter()를-이용해-stopwords(불용어)제거" data-toc-modified-id="-filter()를-이용해-stopwords(불용어)제거-1.2.1">-filter()를 이용해 stopwords(불용어)제거</a></span></li></ul></li><li><span><a href="#3)-foreach()" data-toc-modified-id="3)-foreach()-1.3">3) foreach()</a></span></li><li><span><a href="#4)-pipeline" data-toc-modified-id="4)-pipeline-1.4">4) pipeline</a></span></li></ul></li><li><span><a href="#2.-파일에-쓰기" data-toc-modified-id="2.-파일에-쓰기-2">2. 파일에 쓰기</a></span></li><li><span><a href="#3.-groupBy" data-toc-modified-id="3.-groupBy-3">3. groupBy</a></span><ul class="toc-item"><li><span><a href="#4.-Pair-RDD" data-toc-modified-id="4.-Pair-RDD-3.1">4. Pair RDD</a></span></li></ul></li></ul></div>

## 1. RDD 사용하기
transformation, action 함수 사용하기

In [1]:
import pyspark

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

### 1) map

In [2]:
nRdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nRdd.map(lambda x: x * x)
print (squared)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [3]:
print (squared.collect())

[1, 4, 9, 16]


#### - csv파일에서 읽어 RDD 생성

In [4]:
import os
myRdd4 = spark.sparkContext\
            .textFile(os.path.join("data","ds_spark_2cols.csv"))

In [5]:
myRdd4.take(5)

['35, 2', '40, 27', '12, 38', '15, 31', '21, 1']

- 2차원 리스트로 만들기

map() 함수로 ,로 분리

In [6]:
myRdd5 = myRdd4.map(lambda line: line.split(','))
myRdd5.take(5)

[['35', ' 2'], ['40', ' 27'], ['12', ' 38'], ['15', ' 31'], ['21', ' 1']]

- 정수로 바꾸기

In [7]:
myRdd6 = myRdd5.map(lambda x: [int(i) for i in x])
myRdd6.take(5)

[[35, 2], [40, 27], [12, 38], [15, 31], [21, 1]]

- 단어 분리

In [8]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

map()으로 문장 분리

In [9]:
sentences=myRdd2.map(lambda x:x.split())

In [10]:
sentences.count()

10

- 함수를 만들어 단어 분리

In [11]:
def mySplit(x):
    return x.split()
sentences2=myRdd2.map(mySplit)
sentences2.count()

10

In [12]:
sentences.take(3)

[['Wikipedia'],
 ['Apache',
  'Spark',
  'is',
  'an',
  'open',
  'source',
  'cluster',
  'computing',
  'framework.'],
 ['아파치', '스파크는', '오픈', '소스', '클러스터', '컴퓨팅', '프레임워크이다.']]

- 2차원 배열 출력

In [13]:
for line in sentences.collect():
    for word in line:
        print (word, end=" ")
    print ("\n-----")

Wikipedia 
-----
Apache Spark is an open source cluster computing framework. 
-----
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. 
-----
Apache Spark Apache Spark Apache Spark Apache Spark 
-----
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 
-----
Originally developed at the University of California, Berkeley's AMPLab, 
-----
the Spark codebase was later donated to the Apache Software Foundation, 
-----
which has maintained it since. 
-----
Spark provides an interface for programming entire clusters with 
-----
implicit data parallelism and fault-tolerance. 
-----


In [14]:
myRdd2.map(lambda s:len(s)).collect()

[9, 59, 32, 51, 31, 72, 71, 30, 64, 46]

### 2) filter()로 문장 분리

"Spark"단어가 포함된 문장이 조건

In [15]:
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line)
print ("How many lines having 'Spark': ",myRdd_spark.count())

How many lines having 'Spark':  4


한글을 filter하려면 앞에 u를 붙인다 (u = 유니코드)

In [16]:
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print (myRdd_unicode.first())

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


#### -filter()를 이용해 stopwords(불용어)제거
필요없는 단어 지우기

In [17]:
stopwords = ['is','am','are','the','for','a', 'an', 'at']
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

In [18]:
for words in myRdd_stop.collect():
    print (words, end=' ')

Wikipedia Apache Spark open source cluster computing framework. 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. Apache Spark Apache Spark Apache Spark Apache Spark 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 Originally developed University of California, Berkeley's AMPLab, Spark codebase was later donated to Apache Software Foundation, which has maintained it since. Spark provides interface programming entire clusters with implicit data parallelism and fault-tolerance. 

### 3) foreach()
action이지만 다른 action들과 달리 반환 값이 없다

In [19]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

주피터 노트북에서는 결과 x 파이썬 프럼프트에서 실행

In [20]:
def f(x): print(x)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(f)

### 4) pipeline
transformation(map()), action(collect()) 함수를 연이어 적용

In [21]:
myList=["this is","a line"]
_rdd=spark.sparkContext.parallelize(myList)

In [22]:
wordsRdd=_rdd.map(lambda x:x.split())
print (wordsRdd.collect())

[['this', 'is'], ['a', 'line']]


In [23]:
upper2list=wordsRdd.map(lambda x: [i.upper() for i in x]).collect()
print (type(upper2list))

<class 'list'>


단어의 수 세기

In [24]:
wordsLength = wordsRdd\
    .map(len)\
    .collect()
print (wordsLength)

[2, 2]


## 2. 파일에 쓰기
RDD 저장 Spark에서 list는 RDD로 만들어 로컬 파일에 쓰게 된다

In [25]:
spark.sparkContext.parallelize(upper2list).saveAsTextFile("data/ds_spark_wiki_out")

Py4JJavaError: An error occurred while calling o239.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/Nunu/BigData/data/ds_spark_wiki_out already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:294)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spar

In [26]:
!dir data\ds_spark_wiki_out

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 4E88-AB80

 C:\Users\Nunu\BigData\data\ds_spark_wiki_out 디렉터리

2021-09-22  오후 11:55    <DIR>          .
2021-09-22  오후 11:55    <DIR>          ..
2021-09-22  오후 11:55                12 .part-00000.crc
2021-09-22  오후 11:55                 8 ._SUCCESS.crc
2021-09-22  오후 11:55                29 part-00000
2021-09-22  오후 11:55                 0 _SUCCESS
               4개 파일                  49 바이트
               2개 디렉터리  17,260,871,680 바이트 남음


In [27]:
_rdd=spark.sparkContext.textFile("data/ds_spark_wiki_out")
_rdd.collect()

["['THIS', 'IS']", "['A', 'LINE']"]

In [28]:
_rdd.map(lambda x: "".join(x)).coalesce(1).saveAsTextFile("data/ds_spark_wiki_txt")

Py4JJavaError: An error occurred while calling o263.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/Nunu/BigData/data/ds_spark_wiki_txt already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:294)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spar

In [29]:
!type data\ds_spark_wiki_txt\part-00000

['THIS', 'IS']
['A', 'LINE']


## 3. groupBy
키를 선택하여 사용할 수 있다.

In [30]:
myRdd2.take(10)

['Wikipedia',
 'Apache Spark is an open source cluster computing framework.',
 '아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 '아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크',
 "Originally developed at the University of California, Berkeley's AMPLab,",
 'the Spark codebase was later donated to the Apache Software Foundation,',
 'which has maintained it since.',
 'Spark provides an interface for programming entire clusters with',
 'implicit data parallelism and fault-tolerance.']

앞 2글자를 key로 생성 

In [31]:
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    print ("{}: {}".format(k, v))

Wi: <pyspark.resultiterable.ResultIterable object at 0x000001AC7FFBFC70>
Ap: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700C460>
아파: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700C970>
Or: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700C7F0>
th: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700CC70>
wh: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700C910>
Sp: <pyspark.resultiterable.ResultIterable object at 0x000001AC0700CA60>
im: <pyspark.resultiterable.ResultIterable object at 0x000001AC070866A0>


In [32]:
for (k,v) in myRdd_group.collect():
    for eachValue in v:
        print ("{}: {}".format(k, eachValue))
    print ("-----")

Wi: Wikipedia
-----
Ap: Apache Spark is an open source cluster computing framework.
Ap: Apache Spark Apache Spark Apache Spark Apache Spark
-----
아파: 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
아파: 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
-----
Or: Originally developed at the University of California, Berkeley's AMPLab,
-----
th: the Spark codebase was later donated to the Apache Software Foundation,
-----
wh: which has maintained it since.
-----
Sp: Spark provides an interface for programming entire clusters with
-----
im: implicit data parallelism and fault-tolerance.
-----


groupBy()적용

In [33]:
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]

In [34]:
_testRdd=spark.sparkContext.parallelize(_testList)

In [35]:
_testRdd.groupBy(lambda x:x[0]).collect()

[('Seoul', <pyspark.resultiterable.ResultIterable at 0x1ac7ffab310>),
 ('Busan', <pyspark.resultiterable.ResultIterable at 0x1ac7ff94af0>)]

In [36]:
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

In [37]:
_testRdd.groupBy(lambda x:x[0]).mapValues(list).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

### 4. Pair RDD
key, value 쌍으로 구성된 RDD

연산하는 `byKey()` 또는 값에 대해 `byValue()`를 사용 가능

In [38]:
_testList=[("key1",1),("key1",1),("key1",1),("key2",1),("key2",1),
           ("key1",1),("key2",1),
           ("key1",1),("key1",1),("key2",1),("key2",1)]
_testRdd=spark.sparkContext.parallelize(_testList)

Partition

In [39]:
_testRdd.getNumPartitions()

1

In [40]:
_testRdd=spark.sparkContext.parallelize(_testList, 2)

In [41]:
_testRdd.getNumPartitions()

2

In [42]:
year = 2021
name = 'yanu'
f"Hello, {name} {year}."

'Hello, yanu 2021.'

In [43]:
partitions = _testRdd.glom().collect()
for num, partition in enumerate(partitions):
    print(f'Partitions {num} -> {partition}')

Partitions 0 -> [('key1', 1), ('key1', 1), ('key1', 1), ('key2', 1), ('key2', 1)]
Partitions 1 -> [('key1', 1), ('key2', 1), ('key1', 1), ('key1', 1), ('key2', 1), ('key2', 1)]


- key 출력

In [44]:
_testRdd.keys().collect()

['key1',
 'key1',
 'key1',
 'key2',
 'key2',
 'key1',
 'key2',
 'key1',
 'key1',
 'key2',
 'key2']

reduceByKey 가 병렬적으로 처리하여 더 빠르다

In [45]:
_testRdd.reduceByKey(lambda x,y:x+y).collect()

[('key1', 6), ('key2', 5)]

In [46]:
_testRdd.groupByKey().collect()

[('key1', <pyspark.resultiterable.ResultIterable at 0x1ac07081a90>),
 ('key2', <pyspark.resultiterable.ResultIterable at 0x1ac07081e80>)]

In [47]:
_testRdd.groupByKey().mapValues(list).collect() # list is a function, that is, list()

[('key1', [1, 1, 1, 1, 1, 1]), ('key2', [1, 1, 1, 1, 1])]

In [48]:
_testRdd.mapValues(lambda x:x+1).collect()

[('key1', 2),
 ('key1', 2),
 ('key1', 2),
 ('key2', 2),
 ('key2', 2),
 ('key1', 2),
 ('key2', 2),
 ('key1', 2),
 ('key1', 2),
 ('key2', 2),
 ('key2', 2)]

- 단어 빈도 cnt

In [49]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .take(3)

[('Wikipedia', <pyspark.resultiterable.ResultIterable at 0x1ac0700c100>),
 ('Apache', <pyspark.resultiterable.ResultIterable at 0x1ac0700c9a0>),
 ('Spark', <pyspark.resultiterable.ResultIterable at 0x1ac0700cd00>)]

In [50]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(sum)\
    .take(20)

[('Wikipedia', 1),
 ('Apache', 6),
 ('Spark', 7),
 ('is', 1),
 ('an', 2),
 ('open', 1),
 ('source', 1),
 ('cluster', 1),
 ('computing', 1),
 ('framework.', 1),
 ('아파치', 5),
 ('스파크는', 1),
 ('오픈', 1),
 ('소스', 1),
 ('클러스터', 1),
 ('컴퓨팅', 1),
 ('프레임워크이다.', 1),
 ('스파크', 4),
 ('Originally', 1),
 ('developed', 1)]

In [52]:
def f(x): return len(x)
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(f)\
    .sortByKey(True)\
    .take(10)

[('AMPLab,', 1),
 ('Apache', 6),
 ("Berkeley's", 1),
 ('California,', 1),
 ('Foundation,', 1),
 ('Originally', 1),
 ('Software', 1),
 ('Spark', 7),
 ('University', 1),
 ('Wikipedia', 1)]

In [53]:
wc=myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(sum)\
    .sortByKey(True)\
    .take(10)

In [54]:
for e in wc:
    k = e[0]
    v = e[1]
    print (f"단어:{k}\t\t빈도:{v}")

단어:AMPLab,		빈도:1
단어:Apache		빈도:6
단어:Berkeley's		빈도:1
단어:California,		빈도:1
단어:Foundation,		빈도:1
단어:Originally		빈도:1
단어:Software		빈도:1
단어:Spark		빈도:7
단어:University		빈도:1
단어:Wikipedia		빈도:1


In [55]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .take(10)

[('Wikipedia', 1),
 ('Apache', 6),
 ('Spark', 7),
 ('is', 1),
 ('an', 2),
 ('open', 1),
 ('source', 1),
 ('cluster', 1),
 ('computing', 1),
 ('framework.', 1)]

countByKey

In [56]:
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .countByKey() # .items() to be added to get a list

defaultdict(int,
            {'Wikipedia': 1,
             'Apache': 6,
             'Spark': 7,
             'is': 1,
             'an': 2,
             'open': 1,
             'source': 1,
             'cluster': 1,
             'computing': 1,
             'framework.': 1,
             '아파치': 5,
             '스파크는': 1,
             '오픈': 1,
             '소스': 1,
             '클러스터': 1,
             '컴퓨팅': 1,
             '프레임워크이다.': 1,
             '스파크': 4,
             'Originally': 1,
             'developed': 1,
             'at': 1,
             'the': 3,
             'University': 1,
             'of': 1,
             'California,': 1,
             "Berkeley's": 1,
             'AMPLab,': 1,
             'codebase': 1,
             'was': 1,
             'later': 1,
             'donated': 1,
             'to': 1,
             'Software': 1,
             'Foundation,': 1,
             'which': 1,
             'has': 1,
             'maintained': 1,
             'it': 1,
      